# 📘 20_decorators.ipynb

### 🧩 Topic: Decorators & Closures in Python


## 🧠 1. Why Decorators?
Decorators are a powerful, expressive tool that let you **wrap** a function with extra behavior **without** modifying its source code. They are widely used for logging, timing, access control, caching, and more.

Key ideas:
- Functions are **first-class** (can be passed around)  
- A **decorator** is a callable that takes a function and returns a new function


## 🔹 2. Inner Functions & Closures

In [ ]:
def outer(msg):
    greeting = "Hello " + msg
    def inner():
        # inner function captures `greeting` from outer scope — this is a closure
        print(greeting)
    return inner

fn = outer("World")
fn()  # prints "Hello World" — `inner` retained `greeting` even after outer finished

## 🌀 3. Basic Decorator Syntax

In [ ]:
# Basic decorator that prints before and after a function call
def simple_decorator(func):
    def wrapper(*args, **kwargs):
        print("Before call")
        result = func(*args, **kwargs)
        print("After call")
        return result
    return wrapper

@simple_decorator
def say(name):
    print(f"Hi {name}")

say("Surendra")

## 🔁 4. Decorator with Arguments (Parameterized Decorator)

In [ ]:
def repeat(n):
    def decorator(func):
        def wrapper(*args, **kwargs):
            for _ in range(n):
                func(*args, **kwargs)
        return wrapper
    return decorator

@repeat(3)
def greet(name):
    print("Hello", name)

greet("Akhilesh")

## 🔗 5. Chaining Multiple Decorators

In [ ]:
def deco1(f):
    def wrapper(*a, **k):
        print("deco1 before")
        r = f(*a, **k)
        print("deco1 after")
        return r
    return wrapper

def deco2(f):
    def wrapper(*a, **k):
        print("deco2 before")
        r = f(*a, **k)
        print("deco2 after")
        return r
    return wrapper

@deco1
@deco2
def target():
    print("target running")

target()

## 🧰 6. Preserving Metadata — `functools.wraps`

In [ ]:
import functools

def logged(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print(f"Calling {func.__name__}")
        return func(*args, **kwargs)
    return wrapper

@logged
def add(a, b):
    """Return the sum of a and b"""
    return a + b

print(add(2,3))
print('Function name:', add.__name__)
print('Docstring:', add.__doc__)

## ⚙️ 7. Real-World Examples

**Logging decorator** and **Timing decorator** examples — these are commonly used in production code.

In [ ]:
import time, functools, logging
logging.basicConfig(level=logging.INFO)

def timing(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        print(f"{func.__name__} took {end-start:.6f}s")
        return result
    return wrapper

def logger(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        logging.info(f"Calling {func.__name__} with args={args} kwargs={kwargs}")
        return func(*args, **kwargs)
    return wrapper

@timing
@logger
def compute(n):
    total = 0
    for i in range(n):
        total += i*i
    return total

print(compute(10000))

## 🔒 8. Access Control Decorator (Authentication Example)

In [ ]:
# Simple auth decorator example
def requires_role(role):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(user, *args, **kwargs):
            if getattr(user, 'role', None) != role:
                raise PermissionError(f"User needs role {role}")
            return func(user, *args, **kwargs)
        return wrapper
    return decorator

class User:
    def __init__(self, name, role):
        self.name = name
        self.role = role

@requires_role('admin')
def delete_resource(user, resource_id):
    print(f"{user.name} deleted resource {resource_id}")

admin = User('Surendra', 'admin')
delete_resource(admin, 42)

## 🧩 9. Mini Project — Function Performance Logger Decorator

In [ ]:
# Performance logger that writes to a file
import json, time, functools
LOG_FILE = '/mnt/data/perf_log.jsonl'  # JSON Lines (one JSON per line)

def perf_logger(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        duration = time.perf_counter() - start
        entry = {
            'func': func.__name__,
            'args': args,
            'kwargs': kwargs,
            'duration_s': duration,
            'timestamp': time.time()
        }
        # append JSON line
        with open(LOG_FILE, 'a') as f:
            f.write(json.dumps(entry) + '\n')
        return result
    return wrapper

@perf_logger
def heavy(n):
    s = 0
    for i in range(n):
        s += i*i
    return s

# run demo
heavy(5000)
heavy(10000)

# show last 2 log lines
with open(LOG_FILE) as f:
    lines = f.readlines()[-2:]
    for line in lines:
        print(json.loads(line))

## 💡 10. Tips & Gotchas


- Be careful with decorators that modify call signatures (use `*args, **kwargs`).  
- Use `functools.wraps` to preserve function metadata.  
- Avoid side-effects in decorators when possible.  
- Test decorated functions as you would test normal functions.


## 🧩 11. Beginner-Level Challenges


1️⃣ Write a decorator `@uppercase` that converts the string result of a function to uppercase.  
2️⃣ Write a `@retry` decorator that retries a function up to `n` times if it raises an exception.


In [ ]:
# Example solution: uppercase decorator
def uppercase(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs).upper()
    return wrapper

@uppercase
def say_hi(name):
    return f"hi {name}"

print(say_hi("surendra"))

## 💪 12. Advanced Challenges


1️⃣ Create a caching decorator `@memoize` that caches function results based on arguments.  
2️⃣ Implement an `@async_retry` decorator that works with `async` functions (use `asyncio.sleep` for backoff).  
3️⃣ Create a decorator factory that creates decorators adding timed metrics to Prometheus (conceptual).


In [ ]:
# 1️⃣ Simple memoize decorator (not thread-safe, for demonstration)
def memoize(func):
    cache = {}
    @functools.wraps(func)
    def wrapper(*args):
        if args in cache:
            return cache[args]
        res = func(*args)
        cache[args] = res
        return res
    return wrapper

@memoize
def fib(n):
    if n < 2:
        return n
    return fib(n-1) + fib(n-2)

print(fib(20))

## 🧠 13. Summary


| Concept | Notes |
|--------|-------|
| Closure | Inner function remembers outer state |
| Decorator | Function that returns a wrapper function |
| Parameterized decorator | Decorator factory that accepts arguments |
| functools.wraps | Preserve metadata of wrapped function |
| Use cases | Logging, timing, auth, caching, retries |



---
## ✅ Next Notebook
👉 `21_context_managers.ipynb` — Learn how to use and build `with` context managers (`__enter__`, `__exit__`, `contextlib`).
